# Atelier tokenizer

Dans cet atelier nous alons constuire un tokenizer utilisant l'algorithme Byte Pair Encoding (BPE) tel que ceux utilisés
dans ChatGPT.


## Convertir une chaine de caractères en une séquence d'entiers

D'après la [documentation python](https://docs.python.org/fr/3/library/stdtypes.html#text-sequence-type-str) :

> Les chaînes sont des séquences immuables de points de code Unicode.

Chaque caractère a un numéro, une catégorie, un nom :

In [2]:
import unicodedata

text = "Bonjour 👋"

for char in text:
    print(char, ord(char), unicodedata.category(char), unicodedata.name(char))

B 66 Lu LATIN CAPITAL LETTER B
o 111 Ll LATIN SMALL LETTER O
n 110 Ll LATIN SMALL LETTER N
j 106 Ll LATIN SMALL LETTER J
o 111 Ll LATIN SMALL LETTER O
u 117 Ll LATIN SMALL LETTER U
r 114 Ll LATIN SMALL LETTER R
  32 Zs SPACE
👋 128075 So WAVING HAND SIGN


Les chaines de caractères sont ensuite encodées pour permettre la sauvegarde, la lecture, etc.

Il existe plusieurs types d'encodage. Le plus utilisé est `UTF-8`. Cet encodage représente chaque point de code par une suite de 1 à 4 bytes en fonction de son indice :


| Premier point de code | Dernier code point | Byte 1       | Byte 2   | Byte 3   | Byte 4   |
|-----------------------|--------------------|--------------|----------|----------|----------|
| U+0000 (0)            | U+007F (127)       | **0**yyyzzzz |          |          |          |
| U+0080 (128)          | U+07FF (2047)      | **110**xxxyy | 10yyzzzz |          |          |
| U+0800 (2048)         | U+FFFF (65535)     | **1110**wwww | 10xxxxyy | 10yyzzzz |          |
| U+010000 (65536)      | U+10FFFF (1114111) | **11110**uvv | 10vvwwww | 10xxxxyy | 10yyzzzz |

<br/>

> **Rappel :** Un _byte_ en anglais correspond à un _octet_ en français soit 8 _bits_.
> 
> Un octet peut donc prendre 2^8 = 256 valeurs

In [37]:
text = "Bonjour 👋"

for char in text:
    for byte in char.encode("utf-8"):
        print(char, "|", byte, f"| code hexadécimal : {byte:02x}")

B | 66 | code hexadécimal : 42
o | 111 | code hexadécimal : 6f
n | 110 | code hexadécimal : 6e
j | 106 | code hexadécimal : 6a
o | 111 | code hexadécimal : 6f
u | 117 | code hexadécimal : 75
r | 114 | code hexadécimal : 72
  | 32 | code hexadécimal : 20
👋 | 240 | code hexadécimal : f0
👋 | 159 | code hexadécimal : 9f
👋 | 145 | code hexadécimal : 91
👋 | 139 | code hexadécimal : 8b


On peut directement obtenir la liste des bytes, il s'agit d'une première tokenization :

In [4]:
text = "Bonjour 👋"
text_ids = list(text.encode())

print(text_ids)

[66, 111, 110, 106, 111, 117, 114, 32, 240, 159, 145, 139]


## Implémentation de la tokenization Byte Pair Encoding (BPE)

In [38]:
text = "Bonjour 👋"
text_ids = list(text.encode())
print(text_ids)

[66, 111, 110, 106, 111, 117, 114, 32, 240, 159, 145, 139]


In [ ]:
# Cette fonction permettra de tester les différentes fonctions de l'atelier
from tests import test

[L'algorithme BPE ](https://en.wikipedia.org/wiki/Byte_pair_encoding) fonctionne en fusionnant sucessivement la paire
de tokens la plus fréquente.

Dans un premier temps, on va donc implémenter une fonction qui détermine la paire de tokens la plus fréquente.

In [6]:
text = """Une demi-heure plus tard, Harry, qui n'en croyait pas sa chance, était assis à l'arrière de la voiture des
Dursley, en compagnie de Piers et Dudley. Pour la première fois de sa vie, il allait visiter le zoo."""

text_ids = list(text.encode())
print(text_ids)

[85, 110, 101, 32, 100, 101, 109, 105, 45, 104, 101, 117, 114, 101, 32, 112, 108, 117, 115, 32, 116, 97, 114, 100, 44, 32, 72, 97, 114, 114, 121, 44, 32, 113, 117, 105, 32, 110, 39, 101, 110, 32, 99, 114, 111, 121, 97, 105, 116, 32, 112, 97, 115, 32, 115, 97, 32, 99, 104, 97, 110, 99, 101, 44, 32, 195, 169, 116, 97, 105, 116, 32, 97, 115, 115, 105, 115, 32, 195, 160, 32, 108, 39, 97, 114, 114, 105, 195, 168, 114, 101, 32, 100, 101, 32, 108, 97, 32, 118, 111, 105, 116, 117, 114, 101, 32, 100, 101, 115, 10, 68, 117, 114, 115, 108, 101, 121, 44, 32, 101, 110, 32, 99, 111, 109, 112, 97, 103, 110, 105, 101, 32, 100, 101, 32, 80, 105, 101, 114, 115, 32, 101, 116, 32, 68, 117, 100, 108, 101, 121, 46, 32, 80, 111, 117, 114, 32, 108, 97, 32, 112, 114, 101, 109, 105, 195, 168, 114, 101, 32, 102, 111, 105, 115, 32, 100, 101, 32, 115, 97, 32, 118, 105, 101, 44, 32, 105, 108, 32, 97, 108, 108, 97, 105, 116, 32, 118, 105, 115, 105, 116, 101, 114, 32, 108, 101, 32, 122, 111, 111, 46]


In [ ]:
def get_top_pair(ids: list[int]) -> tuple[tuple[int, int], int] | None:
    """Récupération de la paire de tokens la plus fréquente."""

In [40]:
get_top_pair([1, 2, 1, 2, 3])

((1, 2), 2)

In [8]:
test(get_top_pair)

✅ TEST 0
✅ TEST 1
✅ TEST 2


In [9]:
assert (top_pair := get_top_pair(text_ids))

pair, nb = top_pair
pair_str = bytes(pair).decode("utf-8")

print(f"Paire la plus fréquente : {pair} = ('{pair_str}') avec {nb} apparitions")

Paire la plus fréquente : (101, 32) = ('e ') avec 10 apparitions


Ensuite on va écfire une fonction qui fusionne une paire en remplaçant les octets par une nouvel id.

In [ ]:
def merge(ids: list[int], pair: tuple[int, int], new_id: int) -> list[int]:
    """Fusion d'une pair de tokens."""


merge([1, 2, 3, 1, 2, 3, 3], (2, 3), 4)

[1, 4, 1, 4, 3]

In [11]:
test(merge)

✅ TEST 0
✅ TEST 1
✅ TEST 2
✅ TEST 3


### Entrainement du tokenizer

Les fonctions `get_top_pair` et `merge` peuvent maintenant être utilisée pour entrainer un tokenizer BPE.

Ecrivons une fonction `train_bpe_tokenizer` 


In [ ]:
def train_bpe_tokenizer(text: str, vocab_size: int) -> dict[tuple[int, int], int]:
    """Entrainement d'un tokenizer BPE."""


merges = train_bpe_tokenizer(text, 266)

Nombre de tokens initial : 211
Nombre de tokens final   : 165
Ratio de compression     : 78%


In [13]:
test(train_bpe_tokenizer)

Nombre de tokens initial : 26
Nombre de tokens final   : 9
Ratio de compression     : 35%
✅ TEST 0
Nombre de tokens initial : 0
Nombre de tokens final   : 0
Ratio de compression     : 0%
✅ TEST 1
✅ TEST 2


### Encodage (tokenisation)

A présent on peut tokenizer une chaine de caractère

In [ ]:
def encode(text: str, merges: dict[tuple[int, int], int]) -> list[int]:
    """Tokenization d'une chaine de caractère à partir du dictionnaire merges."""


ids = encode(text, merges)
print(len(ids))
print(ids)

165
[85, 110, 260, 101, 109, 105, 45, 104, 101, 261, 256, 112, 108, 117, 257, 116, 263, 100, 258, 72, 263, 114, 121, 258, 113, 117, 105, 32, 110, 39, 101, 110, 32, 99, 114, 111, 121, 265, 112, 97, 257, 115, 262, 99, 104, 97, 110, 99, 101, 258, 195, 169, 116, 265, 97, 115, 115, 105, 257, 195, 160, 32, 108, 39, 263, 114, 105, 195, 168, 114, 260, 256, 108, 262, 118, 111, 259, 261, 260, 101, 115, 10, 68, 261, 115, 108, 101, 121, 258, 101, 110, 32, 99, 111, 109, 112, 97, 103, 110, 105, 260, 256, 80, 105, 101, 114, 257, 101, 116, 32, 68, 117, 100, 108, 101, 121, 46, 32, 80, 111, 261, 32, 108, 262, 112, 114, 101, 109, 105, 195, 168, 114, 256, 102, 111, 105, 257, 100, 256, 115, 262, 118, 105, 101, 258, 105, 108, 32, 97, 108, 108, 265, 118, 105, 115, 259, 101, 114, 32, 108, 256, 122, 111, 111, 46]


In [15]:
test(encode)

✅ TEST 0
✅ TEST 1
✅ TEST 2


### Décodage

Maintenant on souhaite décoder une séquence de tokens.

Dans un premier temps, on va construire un vocabulaire qui à chaque token, va associé les octets correspondant.

In [ ]:
def construct_vocab(merges: dict[tuple[int, int], int]) -> dict[int, bytes]:
    """Construction d'un vocabulaire à partir d'un dictionnaire merges."""


vocab = construct_vocab(merges)
print(vocab)

{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

In [46]:
test(construct_vocab)

✅ TEST 0
✅ TEST 1


In [ ]:
def decode(tokens: list[int], vocab: dict[int, bytes]) -> str:
    """Décodage d'une séquence de tokens"""


print(decode(ids, vocab))

Une demi-heure plus tard, Harry, qui n'en croyait pas sa chance, était assis à l'arrière de la voiture des
Dursley, en compagnie de Piers et Dudley. Pour la première fois de sa vie, il allait visiter le zoo.


In [50]:
test(decode)

✅ TEST 0
✅ TEST 1
✅ TEST 2
✅ TEST 3


Le texte doit être préservé lorsqu'on l'encode puis le décode :

In [27]:
t = "Bonjour 👋"
assert t == decode(encode(t, merges), vocab)

## Bonus : Tokenization BPE et regex !

Une des limites de l'algorithme BPE est que les mots courants tels que `chien` peuvent tout à faire se retrouver dans
plusieurs tokens avec une ponctuation différente :  `chien`, ` chien`,`chien.`, `chien,`, `chien!`, `chien?`.

Cela est problématique pour les modèles de langage car le modèle doit "comprendre" à partir des textes sur lesquels il
est entrainté que tous ces tokens désigne le même concept.

C'est également un problème pour les nombres car il peut très bien y avoir uniquement des tokens `1`, `11`, `2` ce qui
provoque des tokenization étrange des nombres : 
- `123` -> `1`, `2`, `3`
- `112` -> `11`, `2`

Pour éviter ce genre de phénomène, OpenAI utilise en réalité une variante de l'algorithme BPE qui découpe au préalable
le texte à l'aide d'une regex.


In [ ]:
import re

pattern = re.compile(r""" ?[^\W\d_]+| ?\d{1,3}|\s+(?!\S)|\s+""")


pattern.findall(
    "👋 你好   Bonjour _xXxFortiche123. Vous avez : 123456 € sur votre compte."
)

OpenAI utilise des regex plus compliqués pour ces tokenizers :

```python
# the main GPT text split patterns, see
# https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
```

On peut réécrire les fonctions `count_pairs` et `train_bpe_tokenizer` pour les faire fonctionner sur des séquences de tokens :

In [ ]:
def get_top_pair_sequences(tokens_sequences: list[list[int]]):
    """Récupère la paire la plus fréquente parmi une liste de liste de tokens."""


def train_bpe_tokenizer_sequences(
    text: str, vocab_size: int, pattern: re.Pattern = pattern
):
    """Entrainement d'un tokenizer BPE avec une regex."""


merges = train_bpe_tokenizer_sequences(text, 266)
merges